In [1]:
# import cv2

# print("OpenCV version:", cv2.__version__)

In [2]:
# !pip install opencv-contrib-python

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image 
import numpy as np

# Load the saved model
model = load_model("ie_final_3epo_vgg_sahil.h5")

C:\Users\arush\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [4]:
import cv2

def otsu_bin(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, binary_image = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary_image = cv2.bitwise_not(binary_image)

    return binary_image

In [5]:

def preprocess_and_predict(frame):
    
    img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    img = image.img_to_array(img)

    height, width, channels = img.shape

    target_aspect_ratio = 240 / 195

    original_aspect_ratio = width / height

    if original_aspect_ratio > target_aspect_ratio:
        new_height = height
        new_width = int(height * target_aspect_ratio)
        left = (width - new_width) // 2
        right = left + new_width
        top = 0
        bottom = height
    else:
        new_width = width
        new_height = int(width / target_aspect_ratio)
        top = (height - new_height) // 2
        bottom = top + new_height
        left = 0
        right = width

    img = img[top:bottom, left:right]

    img = image.smart_resize(img, (224, 224))

    x = img / 255.0

    x = np.expand_dims(x, axis=0)

    predictions = model.predict(x)
    
    #class_names = ['call_me', 'okay', 'paper','Rock','up']
    class_names = ['Right', 'Left', 'Stop','Forward','Backward']
    predicted_class_idx = np.argmax(predictions)
    predicted_class = class_names[predicted_class_idx]
    
    return predicted_class
    

In [6]:
# print(preprocess_and_predict("up.jpg"))
# print(preprocess_and_predict("call_me.jpg"))
# print(preprocess_and_predict("okay.jpg"))
# print(preprocess_and_predict("rock.jpg"))
# print(preprocess_and_predict("test_paper.jpg"))


In [7]:

import cv2
import serial
import time

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture("http://192.168.53.124:4747/video")

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Capture video frame by frame
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame was successfully read
    if not ret:
        print("Error: Could not read frame.")
        break
    frame=otsu_bin(frame)
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

# Rotate left (counter-clockwise)
    #frame_rotated_left = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # Display the frame
    window_width = 600  # Adjust this value as per your requirement
    window_height = 600  # Adjust this value as per your requirement
    cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Webcam', window_width, window_height)
    cv2.imshow('Webcam', frame)
    # print(preprocess_and_predict(frame))

    # Check for key press
    key = cv2.waitKey(1)
    if key == ord('p'):  # Press 'q' to quit
        print(preprocess_and_predict(frame))
        
        ################################################
        # Configure the serial port
        ser = serial.Serial('COM3', 9600)  # Change 'COM3' to the appropriate port
        time.sleep(2)  # Allow time for Arduino to reset

        # Send data to Arduino
        if preprocess_and_predict(frame)=='Backward':
            ser.write(b'B\n')
        elif preprocess_and_predict(frame)=='Forward':
                ser.write(b'F\n')
        elif preprocess_and_predict(frame)=='Right':
            ser.write(b'R\n')
        elif preprocess_and_predict(frame)=='Left':
            ser.write(b'L\n')
        elif preprocess_and_predict(frame)=='Stop':
            ser.write(b'S\n')
            
        # Close the serial port
        ser.close()
        ###################################################
        
    elif key== ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows() 


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Forward
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Backward
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
Right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step


In [8]:
# # Path to your input image
# image_path = 'input.jpg'

# # Call the function to get the binary image
# binary_img = otsu_binar(image_path)

# window_width = 600  # Adjust this value as per your requirement
# window_height = 600  # Adjust this value as per your requirement
# cv2.namedWindow('Binary Image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Binary Image', window_width, window_height)

# # Display the binary image
# cv2.imshow('Binary Image', binary_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
# import numpy as np
# from sklearn.cluster import KMeans
# from tensorflow.keras.preprocessing import image
# import matplotlib.pyplot as plt

# def image_binarization(img_path):
#     # Load the image
#     img = image.load_img(img_path)
#     img = image.img_to_array(img)

#     # Flatten the image
#     image_flattened = img.reshape((-1, 1))

#     # Apply K-Means clustering
#     kmeans = KMeans(n_clusters=2)
#     kmeans.fit(image_flattened)

#     # Get the centroids
#     centroids = kmeans.cluster_centers_

#     # Binarize the image
#     binary_image = np.where(img > centroids.mean(), 255, 0).astype(np.uint8)
#     binary_image = binary_image.reshape(img.shape)

#     return binary_image

In [10]:
# # Path to your input image
# img_path = 'input.jpg'

# # Call the function to get the binary image
# binary_img = image_binarization(img_path)

# # Display the original and binary images
# plt.subplot(1, 2, 1), plt.imshow(image.load_img(img_path), cmap='gray'), plt.title('Original Image')
# plt.subplot(1, 2, 2), plt.imshow(binary_img, cmap='gray'), plt.title('Binary Image')
# plt.show()

In [11]:

# def preprocess_and_predict(img_path):
#     img = image.load_img(img_path)
#     img = image.img_to_array(img)

#     height, width, channels = img.shape

#     target_aspect_ratio = 240 / 195

#     original_aspect_ratio = width / height

#     if original_aspect_ratio > target_aspect_ratio:
#         new_height = height
#         new_width = int(height * target_aspect_ratio)
#         left = (width - new_width) // 2
#         right = left + new_width
#         top = 0
#         bottom = height
#     else:
#         new_width = width
#         new_height = int(width / target_aspect_ratio)
#         top = (height - new_height) // 2
#         bottom = top + new_height
#         left = 0
#         right = width

#     img = img[top:bottom, left:right]

#     img = image.smart_resize(img, (224, 224))

#     x = img / 255.0

#     x = np.expand_dims(x, axis=0)

#     predictions = model.predict(x)
    
#     class_names = ['call_me', 'okay', 'paper','Rock','up']
#     predicted_class_idx = np.argmax(predictions)
#     predicted_class = class_names[predicted_class_idx]
#     return predicted_class
    